In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import MyFunctionsList as mfs
from splinter import Browser
from DoNotPublish import password
from sqlalchemy import create_engine
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.78M/6.78M [00:00<00:00, 43.2MB/s]


In [3]:
# HTML to list_of_leagues
html_link = "https://www.transfermarkt.us/wettbewerbe/europa/wettbewerbe?plus=1"

# Go to website
browser.visit(html_link)

# Convert to html >>> SITE = FULL PAGE HTML
site = soup(browser.html, "html.parser")

In [4]:
# Function inputs: site

# Generate List of rows:
# Given that there are burried <tr> in the html which would result result in a list which was difficult to parse,
# here we use the two td/class combinations which always correspond to a league. This makes future cleaning easier. 
first_half_of_league_list = site.find_all("tr", class_="odd")
second_half_of_league_list = site.find_all("tr", class_="even")
leagues = first_half_of_league_list + second_half_of_league_list

# Create list to store all information, league_id
clean_leagues_data = []

# Interate through rows
for league in leagues:
    
    try:
    # Create list to store individual league information
        league_info = []
    
    # Store league name
        league_name = league.find_all("a")
        league_info.append(league_name[1]['title'])

    # General Data Field for use in later data points
        general_data = league.find_all("td", class_="zentriert")
        
    # Store country
        country_field = general_data[0]("img")[0]["title"]
        league_info.append(country_field)
        
    # Store total_clubs
        total_clubs = general_data[1].text.strip()
        league_info.append(total_clubs)
        
    # Store total_players
        total_players = general_data[2].text.strip()
        league_info.append(total_players)
        
    # Store average_age
        average_age = general_data[3].text.strip()
        league_info.append(average_age)
    
    # Store % percent_foreigners_players
        foreign_players = general_data[4].text.strip()
        foreign_players = float(foreign_players.rstrip("%"))
        league_info.append(foreign_players)
        

    # Store foreigners_game_ratio
        foreign_players_game_ratio = general_data[5].text.strip()
        foreign_players_game_ratio = float(foreign_players_game_ratio.rstrip("%"))
        if foreign_players_game_ratio == '':
            foreign_players_game_ratio = 0
        league_info.append(foreign_players_game_ratio)
    
    # Store goals_per_match
        goals_per_match = general_data[6].text.strip()
        if goals_per_match == '':
            goals_per_match = 0
        league_info.append(goals_per_match)
    
    # Market Data Field for use in later data points
        general_data = league.find_all("td", class_="rechts")
        
    # Store average_market_value_euro
        roster_average_market_value = general_data[0].text.strip()
        league_info.append(mfs.number_cleaner(roster_average_market_value))
    
    # Store total_value_euro
        roster_total_market_value = general_data[1].text.strip()
        league_info.append(mfs.number_cleaner(roster_total_market_value))
       
    # Store league_link
        league_link = league.find_all("a")
        league_link_tail = league_link[1]['href']
        league_info.append(f"https://www.transfermarkt.us{league_link_tail}")
    
    # Store League_level
        league_tier = league_link_tail[-1]
        league_info.append(league_tier)
        
    except:
        pass
    
    clean_leagues_data.append(league_info)

In [5]:
# Create dataframe from extracted data, set league_id as index

clean_leagues_dataframe = pd.DataFrame(clean_leagues_data, columns=(
                                                                "league",
                                                                "country",
                                                                "total_clubs",
                                                                "total_players",
                                                                "average_age",
                                                                "percent_foreigners_players",
                                                                "foreigners_game_ratio",
                                                                "goals_per_match",
                                                                "average_league_value_euro",
                                                                "total_league_value_euro",
                                                                "league_link",
                                                                "league_tier"))

In [6]:
# Reorder Columns

clean_leagues_dataframe = clean_leagues_dataframe[["league",
                                                    "country",
                                                    "league_tier",
                                                    "total_clubs",
                                                    "total_players",
                                                    "average_age",
                                                    "percent_foreigners_players",
                                                    "foreigners_game_ratio",
                                                    "goals_per_match",
                                                    "average_league_value_euro",
                                                    "total_league_value_euro",
                                                    "league_link",
                                                    ]]

In [7]:
# Log Into SQL Dataframe
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'european_football_db'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [8]:
# Upload Data to Database
clean_leagues_dataframe.to_sql(name="leagues", con=engine, if_exists="append", index=False)

In [9]:
browser.quit()

In [10]:
clean_leagues_dataframe

,league,country,league_tier,total_clubs,total_players,average_age,percent_foreigners_players,foreigners_game_ratio,goals_per_match,average_league_value_euro,total_league_value_euro,league_link
0,Premier League,England,1,20,534,26.8,68.7,68.3,2.74,515830000,10320000000,https://www.transfermarkt.us/premier-league/st...
1,Serie A,Italy,1,20,562,26.4,62.3,66.2,2.56,226090000,4520000000,https://www.transfermarkt.us/serie-a/startseit...
2,Ligue 1,France,1,20,529,25.9,54.1,57.6,2.87,167590000,3350000000,https://www.transfermarkt.us/ligue-1/startseit...
3,Süper Lig,Turkey,1,19,532,27.1,52.3,61.8,2.95,54640000,1040000000,https://www.transfermarkt.us/super-lig/startse...
4,Jupiler Pro League,Belgium,1,18,489,24.9,59.1,62.3,2.96,45870000,825730000,https://www.transfermarkt.us/jupiler-pro-leagu...
5,Bundesliga,Austria,1,12,351,24.6,39.0,43.9,3.00,34720000,416600000,https://www.transfermarkt.us/bundesliga/starts...
6,Scottish Premiership,Scotland,1,12,316,26.6,59.8,60.7,2.93,29420000,353080000,https://www.transfermarkt.us/scottish-premiers...
7,Super League,Switzerland,1,10,279,25.2,52.7,56.3,2.94,26600000,266050000,https://www.transfermarkt.us/super-league/star...
8,SuperSport HNL,Croatia,1,10,297,25.1,37.0,39.6,2.55,24950000,249480000,https://www.transfermarkt.us/supersport-hnl/st...
9,Super liga Srbije,Serbia,1,16,460,25.2,22.4,22.1,2.60,14530000,232410000,https://www.transfermarkt.us/super-liga-srbije...
